<a href="https://colab.research.google.com/github/sh0wbit01/DriftDataMangement/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
import numpy as np
import pandas as pd
from math import log2
import collections
import json
import random
#import mab
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split



In [235]:
from google.colab import drive

drive.mount('/content/drive')

df=pd.read_csv('/content/drive/MyDrive/motor.csv')

'''
data1 = scipy.io.loadmat('/content/drive/My Drive/sea.mat')
data = data1.get('data')
n_feature = 3
#pd.DataFrame(data)
'''
df=df.drop(df.columns[0], axis=1)
labels=list(df.iloc[:,-1])
labels=pd.DataFrame({'labels':labels})
df=df.drop(df.columns[3],axis=1)


from sklearn.preprocessing import OneHotEncoder



#from sklearn.compose import ColumnTransformer
'''
onehotencoder = OneHotEncoder()
onehotencoder.fit(labels)
onehotlabels = onehotencoder.transform(labels).toarray()
labels=pd.DataFrame(onehotlabels)
labels.reset_index(drop=True,inplace=True)
df.reset_index(drop=True,inplace=True)
data=pd.concat([df,labels],axis=1)
'''


X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.3, random_state=42, shuffle=True)

X_train=np.asarray(X_train)
print(X_train)

y_train=np.asarray(y_train)
print(y_train.dtype)
X_test=np.asarray(X_test)
y_test=np.asarray(y_test)
print(y_test.dtype)

X_train, X_test, y_train, y_test = map(torch.tensor,(X_train, X_test, y_train, y_test))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[ 0.25     1.07031 -0.28125]
 [ 0.03906  1.03906 -0.10156]
 [ 0.04688  1.       0.00781]
 ...
 [ 0.39844  1.03125 -0.1875 ]
 [-0.01562  1.11719 -0.03125]
 [ 0.16406  0.9375  -0.25781]]
int64
int64


In [266]:

class model(nn.Module):
  def __init__(self,features,neurons,max_hidden_layers,classes):
    super(model,self).__init__()

    device='cuda' if torch.cuda.is_available() else 'cpu'
    print('using {} device...'.format(device))

    self.features=features
    self.neurons=neurons
    self.max_hidden_layers=max_hidden_layers
    self.classes=classes
    
    self.hidden_layers = []
    self.output_layers = []
   
    #self.b = Parameter(torch.tensor(
        
        #  b), requires_grad=False).to(self.device)
    
    self.n = Parameter(torch.tensor(
          0.01), requires_grad=False)


    self.hidden_layers.append(nn.Linear(features,neurons))
   
    for i in range(max_hidden_layers-1):
      self.hidden_layers.append(nn.Linear(neurons,neurons))

    for i in range(max_hidden_layers):
      self.output_layers.append(nn.Linear(neurons,classes))

    
    self.hidden_layers=nn.ModuleList(self.hidden_layers).to(device)
    self.output_layers=nn.ModuleList(self.output_layers).to(device)


    self.a=Parameter(torch.Tensor(self.max_hidden_layers).fill_(1/self.max_hidden_layers+1),requires_grad=False)
    print(self.a)

    '''
    def zero_grad(self):
      for i in range(self.max_num_hidden_layers):
          
        self.output_layers[i].weight.grad.data.fill_(0)
        self.output_layers[i].bias.grad.data.fill_(0)
        self.hidden_layers[i].weight.grad.data.fill_(0)
        self.hidden_layers[i].bias.grad.data.fill_(0)
        '''

         
    
 


      
  

  def update(self,x,y):

    predictions=self.forward(x)
   # y=torch.tensor(y,dtype=torch.float64)
#    y=torch.tensor(0.4)
    

    losses=[]

    for prediction in predictions:
      #loss=self.cross_entropy(y,prediction)
    #  print(prediction,y)
      criterion=nn.CrossEntropyLoss()
      loss=criterion(prediction.unsqueeze(0),y)

      losses.append(loss)
    print(losses)


    
    w=[None]*len(losses)
    b=[None]*len(losses)



    for i in range(len(losses)):
      #print(losses[i])
      
      
    #losses[i]=torch.tensor(losses[i])
      losses[i].backward(retain_graph=True)

      
      self.output_layers[i].weight.data= -self.n * self.a[i] * self.output_layers[i].weight.grad.data
      self.output_layers[i].bias.data= -self.n * self.a[i] * self.output_layers[i].bias.grad.data

      #print(losses)
      #print(self.output_layers[i])

    
      for j in range(i+1):

        if w[j] is None:


          w[j]=self.a[i] * self.hidden_layers[j].weight.grad.data
          b[j]=self.a[i] * self.hidden_layers[j].weight.grad.data
      
        else:

          w[j]+=self.a[i] * self.hidden_layers[j].weight.grad.data
          b[j]+=self.a[i] * self.hidden_layers[j].weight.grad.data

    
    for i in range(len(losses)):
      self.hidden_layers[i].weight.data= -self.n * w[i]
      self.hidden_layers[i].bias.data= -self.n * b[i]

 #   print(self.hidden_layers)


    self.zero_grad()

    








    
 

  def cross_entropy(self,p, q):

	  return -sum([p[i]*log2(q[i]) for i in range(len(p))])





  
  def forward(self,x):


    hidden_connections = []
    x=torch.tensor(x)
   # print(x)
    

    
    m=nn.Softmax(dim=0)

    X=torch.sigmoid(self.hidden_layers[0](x))
    hidden_connections.append(X)

    for i in range(1, self.max_hidden_layers):
      hidden_connections.append(torch.sigmoid(self.hidden_layers[i](hidden_connections[i-1])))

 

   # print(hidden_connections)

    output_classes=[]
    for i in range(self.max_hidden_layers):
    
      output_classes.append(self.output_layers[i](hidden_connections[i]))

      
      output_classes[i]=m(output_classes[i])

     # print("outputs{}".format(output_classes[i]))

    #print(torch.stack(output_classes))
    
    predictions=torch.stack(output_classes)
    print(predictions)
    
  

    return predictions


  def fit(self,x,y):
    print(x.dtype,y.dtype)
    self.update(x,y)











            
        


   


 
    
    

    
      



    

    






In [267]:


model=model(3,10,5,4)
model.double()
#model.forward(X_train[0])
model.fit(X_train[0],y_train[0])

using cpu device...
Parameter containing:
tensor([1.2000, 1.2000, 1.2000, 1.2000, 1.2000])
torch.float64 torch.int64
tensor([[0.1700, 0.1749, 0.3612, 0.2940],
        [0.1531, 0.2340, 0.4070, 0.2058],
        [0.2482, 0.1530, 0.2757, 0.3231],
        [0.3610, 0.2463, 0.1725, 0.2202],
        [0.1855, 0.2954, 0.3605, 0.1586]], dtype=torch.float64,
       grad_fn=<StackBackward>)
[tensor(1.3456, dtype=torch.float64, grad_fn=<NllLossBackward>), tensor(1.4351, dtype=torch.float64, grad_fn=<NllLossBackward>), tensor(1.3151, dtype=torch.float64, grad_fn=<NllLossBackward>), tensor(1.4185, dtype=torch.float64, grad_fn=<NllLossBackward>), tensor(1.4811, dtype=torch.float64, grad_fn=<NllLossBackward>)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:147: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [269]:
list(model.parameters())

[Parameter containing:
 tensor(0.0100, dtype=torch.float64), Parameter containing:
 tensor([1.2000, 1.2000, 1.2000, 1.2000, 1.2000], dtype=torch.float64), Parameter containing:
 tensor([[-2.3722e-04, -1.0156e-03,  2.6688e-04],
         [ 5.7058e-05,  2.4428e-04, -6.4191e-05],
         [ 3.3249e-05,  1.4235e-04, -3.7405e-05],
         [-9.3203e-05, -3.9902e-04,  1.0485e-04],
         [-2.3245e-05, -9.9516e-05,  2.6150e-05],
         [ 8.5412e-05,  3.6567e-04, -9.6088e-05],
         [ 3.8927e-04,  1.6666e-03, -4.3793e-04],
         [ 1.9559e-04,  8.3735e-04, -2.2003e-04],
         [-7.8648e-05, -3.3671e-04,  8.8479e-05],
         [ 1.8835e-05,  8.0636e-05, -2.1189e-05]], dtype=torch.float64,
        requires_grad=True), Parameter containing:
 tensor([[-2.3722e-04, -1.0156e-03,  2.6688e-04],
         [ 5.7058e-05,  2.4428e-04, -6.4191e-05],
         [ 3.3249e-05,  1.4235e-04, -3.7405e-05],
         [-9.3203e-05, -3.9902e-04,  1.0485e-04],
         [-2.3245e-05, -9.9516e-05,  2.6150e-05],


In [ ]:


#model.state_dict
#model.named_modules


In [198]:



#list(model.parameters())
